# 필요한 라이브러리 설치 및 불러오기

In [ ]:
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# 강화학습 환경 및 에이전트 설정

In [ ]:
class LotteryEnv:
    def __init__(self, win_prob=1/125):
        self.win_prob = win_prob
        self.reset()

    def reset(self):
        self.attempts = 0
        return self.attempts

    def step(self, action):
        reward = 0
        done = False

        if action == 1:  # 시도를 선택
            self.attempts += 1
            if np.random.rand() < self.win_prob:
                reward = 1  # 당첨
                done = True
            else:
                reward = -0.01  # 비당첨
        return self.attempts, reward, done

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # 할인율
        self.epsilon = 1.0   # 탐험률
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# 모델 학습

In [ ]:
env = LotteryEnv()
agent = DQNAgent(state_size=1, action_size=2)  # 상태 크기: 1, 행동 크기: 2 (시도 or 중단)

episodes = 1000  # 총 에피소드 수

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, 1])

    for time in range(500):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, 1])
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            if isinstance(reward, int):
                print("에피소드: {}/{}, 시도: {}, 보상: {}".format(e, episodes, time, reward))
            else:
                print("에피소드: {}/{}, 시도: {}, 보상: {:.2f}".format(e, episodes, time, reward))
            break

    if len(agent.memory) > 32:
        agent.replay(32)


In [ ]:
def test_agent(agent, env, n_tests):
    total_rewards = 0
    for _ in range(n_tests):
        state = env.reset()
        state = np.reshape(state, [1, 1])
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, 1])
            state = next_state
            if done:
                total_rewards += reward

    average_reward = total_rewards / n_tests
    print(f"평균 보상: {average_reward:.2f}")

# 테스트 환경 설정
n_tests = 100  # 테스트할 게임 수
test_agent(agent, env, n_tests)
